# Scraping weather data 


This notebook scrap the historical daily weather information from https://www.wunderground.com/history/daily/. The procedure is the following.

1. Select range of date and for each day put the day into get_data_n_soup function.

2. Inside the get_data_n_soup 
    - Ask Selenium to open the website, and execute inner javascript.
    - Parse data into beautifulsoup object and look for the hourly table
    - Parse the table into a panda dataframe
    - remove the unit 
    - add date column
    
    
3. Use beautifulsoup to extract the unit in get_unit function.  
4. Add unit into Pandas columns name
5. Combine each df from daily table into a single dataframe and save it.

Reference: 
- http://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Scraping_a_Webpage_Rendered_by_Javascript_Using_Python.php
- https://automatetheboringstuff.com/chapter11/

In [1]:
import sys
sys.path.append(r'C:\Users\Benny\fastai\old')
from pathlib import Path
from fastai.imports import *
import requests
import wget
from bs4 import BeautifulSoup
from selenium import webdriver 
from datetime import datetime, date
import time

# Testing section

In [ ]:
browser = webdriver.Chrome() 

In [55]:
# select date range 
start_date = datetime(2016, 11, 21)
stop_date = datetime(2016, 12, 31)
# convert date-time object to string
date_range = pd.date_range(start_date, stop_date).strftime('%Y-%m-%d')
#select one date
date_str = date_range[0]

In [56]:
url=f'thtps://www.wunderground.com/history/daily/th/bang-phut/VTBD/date/{date_str}'
# open the website
browser.get(url)
time.sleep(10)
# execute inner html 
innerhtml= browser.execute_script("return document.body.innerHTML")
# parse html into beautifulsoup object
soup = BeautifulSoup(innerhtml)
# find the hourly weather report  
div_table=soup.find_all(attrs={"id": "history-observation-table"})[0]
#parse into a panda dataframe
daily_df = pd.read_html(str(div_table))[0]

In [57]:
daily_df = df[0]
daily_df.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Precip Accum,Condition
0,12:00 AM,84,79,84,SSW,10,0,29.8,0.0,0.0,Fair
1,12:30 AM,82,79,89,SSW,9,0,29.8,0.0,0.0,Fair
2,1:00 AM,82,79,89,SSW,8,0,29.8,0.0,0.0,Fair
3,1:30 AM,82,81,94,S,8,0,29.8,0.0,0.0,Fair
4,2:00 AM,82,79,89,S,8,0,29.8,0.0,0.0,Fair


In [58]:
# add unit into header

trow = div_table.tbody.find_all('tr')

 #stripping unit
f_row = trow[0]
units=[]
for unit_cell in f_row.find_all(class_="wu-label"):
    unit = ''.join(string for string in unit_cell.stripped_strings)
    units.append(r'('+unit+r')')
units.insert(0, '')
units.insert(4, '')
units.append('')

In [59]:
daily_df.columns + units

Index(['Time', 'Temperature(F)', 'Dew Point(F)', 'Humidity(%)', 'Wind',
       'Wind Speed(mph)', 'Wind Gust(mph)', 'Pressure(in)', 'Precip.(in)',
       'Precip Accum(in)', 'Condition'],
      dtype='object')

In [40]:
daily_df.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Precip Accum,Condition
0,12:00 AM,84,79,84,SSW,10,0,29.8,0.0,0.0,Fair
1,12:30 AM,82,79,89,SSW,9,0,29.8,0.0,0.0,Fair
2,1:00 AM,82,79,89,SSW,8,0,29.8,0.0,0.0,Fair
3,1:30 AM,82,81,94,S,8,0,29.8,0.0,0.0,Fair
4,2:00 AM,82,79,89,S,8,0,29.8,0.0,0.0,Fair


In [39]:
# clean up data
daily_df.iloc[:,1] = daily_df.iloc[:,1].str.replace(' F','')
daily_df.iloc[:,2] = daily_df.iloc[:,2].str.replace(' F','')
daily_df.iloc[:,3] = daily_df.iloc[:,3].str.replace(' %','')
daily_df.iloc[:,5] = daily_df.iloc[:,5].str.replace(' mph','')
daily_df.iloc[:,6] = daily_df.iloc[:,6].str.replace(' mph','')
daily_df.iloc[:,7] = daily_df.iloc[:,7].str.replace(' in','')
daily_df.iloc[:,8] = daily_df.iloc[:,8].str.replace(' in','')
daily_df.iloc[:,9] = daily_df.iloc[:,9].str.replace(' in','')

In [41]:
weather = pd.DataFrame()
weather = pd.concat([weather,daily_df])

In [42]:
weather.head()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Precip Accum,Condition
0,12:00 AM,84,79,84,SSW,10,0,29.8,0.0,0.0,Fair
1,12:30 AM,82,79,89,SSW,9,0,29.8,0.0,0.0,Fair
2,1:00 AM,82,79,89,SSW,8,0,29.8,0.0,0.0,Fair
3,1:30 AM,82,81,94,S,8,0,29.8,0.0,0.0,Fair
4,2:00 AM,82,79,89,S,8,0,29.8,0.0,0.0,Fair


# Action 

put the above code into a function and mass scrape 

In [2]:
import sys
sys.path.append(r'C:\Users\Benny\fastai\old')
from pathlib import Path
from fastai.imports import *
import requests
import wget
from bs4 import BeautifulSoup
from selenium import webdriver 
from datetime import datetime, date
import time

In [10]:
browser = webdriver.Chrome() 

In [14]:
# define date range
start_date = datetime(2016, 12, 21)
stop_date = datetime(2016, 12, 31)
date_range = pd.date_range(start_date, stop_date).strftime('%Y-%m-%d')

In [15]:
weather_2016 = pd.DataFrame()

for date in date_range:
    # obtain daily weather dataframe
    daily_df, div_table = get_data_n_soup(date)
    # get the unit
    units = get_unit(div_table)
    # add the unit into the column names
    daily_df.columns = daily_df.columns+units
    
    print(f'Finish obtaining data on {date}')
    #combine the weather for each day
    weather_2016 = pd.concat([weather_2016,daily_df], axis=0, join='outer')
    print(len(weather_2016))
    
#save weather dataframe
weather_2016.to_csv('data/weather/weather_2016v2.csv')


Finish obtaining data on 2016-12-21
48
Finish obtaining data on 2016-12-22
96
Finish obtaining data on 2016-12-23
144
Finish obtaining data on 2016-12-24
192
Finish obtaining data on 2016-12-25
240
Finish obtaining data on 2016-12-26
287
Finish obtaining data on 2016-12-27
335
Finish obtaining data on 2016-12-28
383
Finish obtaining data on 2016-12-29
430
Finish obtaining data on 2016-12-30
478
Finish obtaining data on 2016-12-31
526


In [13]:
#sometimes the for loop stop in the middle, so save the obtained data here
weather_2016.to_csv('data/weather/weather_2016v1.csv')

In [3]:
def get_data_n_soup(date_str):
    ''' Input: date in string
    - Ask Selenium to open the website, and execute inner javascript.
    - Parse data into beautifulsoup object and look for the hourly table
    - Parse the table into a panda dataframe
    - remove the unit 
    - add date column
    
    return: daily weather dataframe and beauitfulsoup object of that table
    '''
    url=f'https://www.wunderground.com/history/daily/th/bang-phut/VTBD/date/{date_str}'
    browser.get(url)
    time.sleep(10)
    innerhtml= browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerhtml)
    div_table=soup.find_all(attrs={"id": "history-observation-table"})[0]
    daily_df = pd.read_html(str(div_table))[0]
    
    # clean up data
    daily_df.iloc[:,1] = daily_df.iloc[:,1].str.replace(' F','')
    daily_df.iloc[:,2] = daily_df.iloc[:,2].str.replace(' F','')
    daily_df.iloc[:,3] = daily_df.iloc[:,3].str.replace(' %','')
    daily_df.iloc[:,5] = daily_df.iloc[:,5].str.replace(' mph','')
    daily_df.iloc[:,6] = daily_df.iloc[:,6].str.replace(' mph','')
    daily_df.iloc[:,7] = daily_df.iloc[:,7].str.replace(' in','')
    daily_df.iloc[:,8] = daily_df.iloc[:,8].str.replace(' in','')
    daily_df.iloc[:,9] = daily_df.iloc[:,9].str.replace(' in','')
    
    # add date columns
    daily_df['date'] = pd.to_datetime(date + ' ' +daily_df['Time'], format="%Y-%m-%d %I:%M %p")
    return daily_df, div_table

In [4]:
def get_unit(div_table):
    ''' stripe unit from the weather table
    columns that do not have unit get empty string 
    '''
    trow = div_table.tbody.find_all('tr')
    #stripping unit
    f_row = trow[0]
    units=[]
    for unit_cell in f_row.find_all(class_="wu-label"):
        unit = ''.join(string for string in unit_cell.stripped_strings)
        units.append(r'('+unit+r')')
    units.insert(0, '')
    units.insert(4, '')
    units.append('')
    units.append('')
    return units